With acknowledgement to Mayank Tripathi https://github.com/mayank408

In [0]:
import string
import pprint

import pandas as pd



In [0]:
documents = ['Klonopin 0.25 mg po every evening, Fluconazole 200 mg po daily, Synthroid 125 mcg po every day',
             'she will not consider switching to clozapine',
             'lovastatin 40 mg one half tab po daily, multivitamin daily, metformin 500 mg one tab po twice a day',
             'Aspirin 81 mg po once daily, Zoloft 25 mg po once daily, Calcium with vitamin D two tablets po once daily']


In [0]:
normalised_documents = []
for i in documents:
    no_punctuation = ''.join(c for c in i if c not in string.punctuation)
    normalised_documents.append(no_punctuation.lower())
    
for i in normalised_documents:
  print(i)

In [0]:
bows = []
for i in normalised_documents:
    bows.append(i.split(' '))

print(bows)

In [0]:
word_set = set()
for i in bows:
  word_set = word_set.union(set(i))
print(word_set)

In [0]:
wordCounts = []
for i in bows:
  thisWordCount = dict.fromkeys(word_set, 0)
  for word in i:
    thisWordCount[word]+=1
  wordCounts.append(thisWordCount)
 

In [0]:
pd.DataFrame(wordCounts)

In [0]:
def computeTF(wordCount, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordCount.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [0]:
termFreqs = []
for i in range(0,len(bows)): 
  termFreqs.append(computeTF(wordCounts[i],bows[i]))

pd.DataFrame(termFreqs) 

In [0]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [0]:
idfs = computeIDF(wordCounts)
pp.pprint(idfs)

In [0]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [0]:
tfidfs = []
for i in termFreqs:
  tfidfs.append(computeTFIDF(i, idfs))
  

  
pd.DataFrame(tfidfs)


In [0]:
pd.DataFrame(termFreqs)